In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('SpamDetector').getOrCreate()

In [23]:
data = spark.read.option('header', 'true').csv('spam.csv', inferSchema=True)
sms = data.select(data.columns[:2])

In [24]:
sms.printSchema()

root
 |-- v1: string (nullable = true)
 |-- v2: string (nullable = true)



In [25]:
sms.show(3)

+----+--------------------+
|  v1|                  v2|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
+----+--------------------+
only showing top 3 rows



In [26]:
sms = sms.withColumnRenamed("v1","label").withColumnRenamed("v2","text")
sms.show(2)

+-----+--------------------+
|label|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
+-----+--------------------+
only showing top 2 rows



In [36]:
# Import the necessary functions
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import Tokenizer

# Remove punctuation (REGEX provided) and numbers
wrangled = sms.withColumn('text', regexp_replace(sms.text, '[_():;,.!?\\-]', ' '))
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, '[0-9]', ' '))

# Merge multiple spaces
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, ' +', ' '))

# Split the text into words
wrangled = Tokenizer(inputCol='text', outputCol='words').transform(wrangled)

wrangled.show(1, truncate=True)

+-----+--------------------+--------------------+
|label|                text|               words|
+-----+--------------------+--------------------+
|  ham|Go until jurong p...|[go, until, juron...|
+-----+--------------------+--------------------+
only showing top 1 row



In [37]:
from pyspark.ml.feature import StopWordsRemover, HashingTF, IDF

# Remove stop words.
wrangled = StopWordsRemover(inputCol='words', outputCol='terms').transform(wrangled)

# Apply the hashing trick
wrangled = HashingTF(inputCol='terms', outputCol='hash', numFeatures=1024).transform(wrangled)

In [38]:
wrangled.show(3)

+-----+--------------------+--------------------+--------------------+--------------------+
|label|                text|               words|               terms|                hash|
+-----+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|[go, until, juron...|[go, jurong, poin...|(1024,[12,171,191...|
|  ham|Ok lar Joking wif...|[ok, lar, joking,...|[ok, lar, joking,...|(1024,[3,493,565,...|
| spam|Free entry in a w...|[free, entry, in,...|[free, entry, wkl...|(1024,[16,24,35,5...|
+-----+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [41]:
type(wrangled.hash)

pyspark.sql.column.Column

In [40]:
# Convert hashed symbols to TF-IDF
tf_idf = IDF(inputCol='hash', outputCol='features').fit(wrangled).transform(wrangled)
      
#tf_idf.select('terms', 'features').show(4, truncate=False)

Py4JJavaError: An error occurred while calling o468.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 31.0 failed 1 times, most recent failure: Lost task 0.0 in stage 31.0 (TID 31) (172.20.10.30 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (Tokenizer$$Lambda$2941/0x00000008018a1ba8: (string) => array<string>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:136)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:759)
	at scala.collection.Iterator$$anon$10.

In [ ]:
# Split the data into training and testing sets
sms_train, sms_test = sms.randomSplit([0.8, 0.2], seed=13)

# Fit a Logistic Regression model to the training data
logistic = LogisticRegression(regParam=0.2).fit(sms_train)

# Make predictions on the testing data
prediction = logistic.transform(sms_test)                      

# Create a confusion matrix, comparing predictions to known labels
prediction.groupBy('label', 'prediction').count().show()

# Accuracy=(TN+TP)/(TN+TP+FN+FP)—proportionofcorrectpredictions.